In [0]:
!pip install keras_metrics
!pip install extra-keras-metrics
!pip install keras-self-attention

In [0]:
import keras_metrics as km
import extra_keras_metrics as ekm


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/dataset/

/content/drive/My Drive/dataset


In [0]:
ls

deft_eval/  EEG/  images.zip  OLID/


In [0]:
import pandas as pd
import glob
import string
import re
import numpy as np

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
train_files = glob.glob('deft_eval/train/' + "*.deft")
li = []
for filename in train_files:
    df = pd.read_csv(filename, sep='\t',index_col=None, header=None,names=['sentence','label'])
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
y_train=frame['label']
corpus_train = []
for j in frame['sentence']:
    df=j.lower()
    df=df.translate(str.maketrans('', '', string.punctuation))
    df=df.replace( 'link ','')
    df=''.join([i for i in df if not i.isdigit()]) 
    df=" ".join(df.split())
    df = [lemmatizer.lemmatize(token) for token in df.split(" ")]
    df=[word for word in df if not word in stop_words]
    df=" ".join(df)
    corpus_train.append(df)

In [0]:
len(corpus_train),corpus_train[0],y_train[0]

(16659,
 'science includes diverse field astronomy biology computer science geology logic physic chemistry mathematics',
 0)

In [0]:
dev_files = glob.glob('deft_eval/dev/' + "*.deft")
li = []
for filename in dev_files:
    df = pd.read_csv(filename, sep='\t',index_col=None, header=None,names=['sentence','label'])
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
y_dev=frame['label']
corpus_dev = []
for j in frame['sentence']:
    df=j.lower()
    df=df.translate(str.maketrans('', '', string.punctuation))
    df=df.replace( 'link ','')
    df=''.join([i for i in df if not i.isdigit()]) 
    df=" ".join(df.split())
    df = [lemmatizer.lemmatize(token) for token in df.split(" ")]
    df=[word for word in df if not word in stop_words]
    df=" ".join(df)
    corpus_dev.append(df)

In [0]:
len(corpus_dev),corpus_dev[0],y_dev[0]

(810,
 'becomes clear definition application scientific method play major role science',
 0)

In [0]:
import collections
collections.Counter(y_train)

Counter({0: 11090, 1: 5569})

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 300
tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(corpus_train)
sequences = tokenize.texts_to_sequences(corpus_train)
X_train = sequence.pad_sequences(sequences,maxlen=max_len)
word_index = tokenize.word_index

In [0]:
import os
embeddings_index = {}
f = open('OLID/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

embedding_dim = 100

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

10001


In [0]:
test_sequences = tokenize.texts_to_sequences(corpus_dev)
X_dev = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,Dense,Dropout,Bidirectional,SpatialDropout1D,GlobalMaxPool1D
from keras.optimizers import RMSprop,Adam
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
import tensorflow as tf
import keras_metrics as km
from keras_self_attention import SeqSelfAttention


from keras.initializers import Constant
tf.logging.set_verbosity(tf.logging.ERROR)

model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=max_len,
                    trainable=True))

model.add((CuDNNGRU(30,return_sequences=True)))
model.add(SeqSelfAttention(attention_width=20,attention_activation='relu',attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL))

model.add(GlobalMaxPool1D())
model.add(Dense(30, activation="sigmoid"))
model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer=Adam(0.0001), metrics=[km.binary_f1_score()])

class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          1000100   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 300, 30)           11880     
_________________________________________________________________
seq_self_attention_1 (SeqSel (None, 300, 30)           901       
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [0]:
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=5,restore_best_weights=True)

In [0]:
model.fit(X_train,y_train,batch_size=30,epochs=50,verbose=2,class_weight=class_weights,validation_data=(X_dev,y_dev),callbacks=[es])

Train on 16659 samples, validate on 810 samples
Epoch 1/50
 - 31s - loss: 0.7444 - f1_score: 0.4098 - val_loss: 0.6901 - val_f1_score: 0.5215
Epoch 2/50
 - 29s - loss: 0.7133 - f1_score: 0.4360 - val_loss: 0.6668 - val_f1_score: 0.5402
Epoch 3/50
 - 29s - loss: 0.6749 - f1_score: 0.4902 - val_loss: 0.6278 - val_f1_score: 0.5728
Epoch 4/50
 - 28s - loss: 0.6364 - f1_score: 0.5500 - val_loss: 0.5944 - val_f1_score: 0.5867
Epoch 5/50
 - 28s - loss: 0.6098 - f1_score: 0.5942 - val_loss: 0.5753 - val_f1_score: 0.6149
Epoch 6/50
 - 28s - loss: 0.5886 - f1_score: 0.6211 - val_loss: 0.5721 - val_f1_score: 0.6168
Epoch 7/50
 - 28s - loss: 0.5681 - f1_score: 0.6392 - val_loss: 0.5539 - val_f1_score: 0.6185
Epoch 8/50
 - 28s - loss: 0.5547 - f1_score: 0.6468 - val_loss: 0.5468 - val_f1_score: 0.6103
Epoch 9/50
 - 28s - loss: 0.5375 - f1_score: 0.6661 - val_loss: 0.5412 - val_f1_score: 0.6221
Epoch 10/50
 - 28s - loss: 0.5199 - f1_score: 0.6791 - val_loss: 0.5354 - val_f1_score: 0.6285
Epoch 11/50

In [0]:

print(model.evaluate(X_dev,y_dev))

810/810 [==============================] - 0s 588us/step
[0.6243823968333962, 0.6942148262790828]


In [0]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_dev, batch_size=30, verbose=1)

print(classification_report(y_dev, y_pred.round()))

810/810 [==============================] - 1s 655us/step
              precision    recall  f1-score   support

           0       0.87      0.77      0.82       537
           1       0.63      0.77      0.69       273

    accuracy                           0.77       810
   macro avg       0.75      0.77      0.76       810
weighted avg       0.79      0.77      0.78       810



In [0]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_dev,y_pred.round())


array([[415, 122],
       [ 63, 210]])

In [0]:
import keras
keras.backend.clear_session()